In [1]:
# Import dependecies
import pandas as pd
import numpy as np
import difflib
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies_df = pd.read_csv('../data/movies.csv')
movies_df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [4]:
movies_df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [5]:
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director', 'production_companies']

In [6]:
# Replacing null values with null string
for feature in selected_features:
    movies_df[feature] = movies_df[feature].fillna('')


In [7]:
# combining all the selected_features
combined_df = movies_df['genres'] + movies_df['keywords'] + movies_df['tagline'] + movies_df['cast'] + movies_df['director'] + movies_df['production_companies']
combined_df

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

In [8]:
vectorizer = TfidfVectorizer()

In [9]:
feature_vectors = vectorizer.fit_transform(combined_df)
print(feature_vectors)

  (0, 5300)	0.06658757622152417
  (0, 5506)	0.07719799906673884
  (0, 14900)	0.10006023738585378
  (0, 29328)	0.08429846010963403
  (0, 15290)	0.2323090586057348
  (0, 10029)	0.18492298683739533
  (0, 16317)	0.13828194096156365
  (0, 30556)	0.28384313631066155
  (0, 34479)	0.11065020650212594
  (0, 10280)	0.2032544217247949
  (0, 30393)	0.2323090586057348
  (0, 32012)	0.05976314733592446
  (0, 35360)	0.10525869838053162
  (0, 25214)	0.07178881654153738
  (0, 25680)	0.18084621464621511
  (0, 28984)	0.1325320943157888
  (0, 35384)	0.1991776495336147
  (0, 35803)	0.16771711203655737
  (0, 28947)	0.18280612374528368
  (0, 30031)	0.17141335003368377
  (0, 34703)	0.16659155194999575
  (0, 30947)	0.13645730924289162
  (0, 21140)	0.18974478492108335
  (0, 23447)	0.13314716087943707
  (0, 28145)	0.2323090586057348
  :	:
  (4801, 18861)	0.3167733394535455
  (4802, 24545)	0.05842367547021673
  (4802, 19078)	0.05842367547021673
  (4802, 14163)	0.06321137598988995
  (4802, 15612)	0.0587922955441139

In [10]:
# Cosine Similarity: getting the similarity confidence value
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.0683712  0.03194165 ... 0.02178793 0.         0.02108254]
 [0.0683712  1.         0.03873024 ... 0.02164503 0.         0.01241619]
 [0.03194165 0.03873024 1.         ... 0.01367098 0.         0.01068244]
 ...
 [0.02178793 0.02164503 0.01367098 ... 1.         0.         0.01118443]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.02108254 0.01241619 0.01068244 ... 0.01118443 0.         1.        ]]


In [11]:
# Getting input from user
movie_name = 'Avatar'

In [12]:
movie_name

'Avatar'

In [13]:
# Create a list with all the list of the movie names given in the dataset
list_of_all_titles = movies_df['title'].tolist()

In [14]:
find_close_matches = difflib.get_close_matches(movie_name, list_of_all_titles)
find_close_matches

['Avatar']

In [15]:
close_match = find_close_matches[0]

In [16]:
idx_of_movie = movies_df[movies_df.title == close_match]['index'].values[0]
idx_of_movie

0

In [17]:
similarity_score = list(enumerate(similarity[idx_of_movie]))
similarity_score

[(0, 1.0000000000000002),
 (1, 0.06837119876041155),
 (2, 0.03194164589508735),
 (3, 0.041447180054652095),
 (4, 0.08575806499905199),
 (5, 0.041554071657592756),
 (6, 0.020208845220078424),
 (7, 0.03689703941385716),
 (8, 0.03932821945256072),
 (9, 0.07720584923062851),
 (10, 0.07071919941329155),
 (11, 0.017933267713821067),
 (12, 0.036575850338872666),
 (13, 0.04411531296052575),
 (14, 0.07795126173380046),
 (15, 0.042187277986551),
 (16, 0.025660356080099506),
 (17, 0.030155821025443865),
 (18, 0.05398629421676137),
 (19, 0.04799531433461094),
 (20, 0.042422292867873065),
 (21, 0.03653790933013847),
 (22, 0.03947859356320582),
 (23, 0.12464045052965797),
 (24, 0.07689943495959267),
 (25, 0.20653379598403784),
 (26, 0.06354229413584972),
 (27, 0.049401451579855454),
 (28, 0.05448703338059505),
 (29, 0.020944296709237385),
 (30, 0.04098318143085941),
 (31, 0.047541544434922665),
 (32, 0.04935057267969055),
 (33, 0.20702549557257102),
 (34, 0.01422877698634),
 (35, 0.0657463782796532)

In [18]:
sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)
sorted_similar_movies

[(0, 1.0000000000000002),
 (101, 0.262588293888675),
 (335, 0.2623536174468185),
 (1652, 0.2618275749756686),
 (3158, 0.2587308830078751),
 (843, 0.2498989036606999),
 (292, 0.24091351254487164),
 (215, 0.24015791602415704),
 (261, 0.22547691617331306),
 (122, 0.2192431104792305),
 (192, 0.21087357890120517),
 (33, 0.20702549557257102),
 (25, 0.20653379598403784),
 (1124, 0.20367226996242474),
 (587, 0.19842060980257525),
 (838, 0.19746151782656027),
 (341, 0.18962317760826097),
 (2403, 0.1885736793200489),
 (1275, 0.18825723628714666),
 (588, 0.18780019630327327),
 (282, 0.1840884023219569),
 (216, 0.1836821419331562),
 (942, 0.182099548044062),
 (243, 0.17817743516720164),
 (1834, 0.17697144499025605),
 (251, 0.17173586573002303),
 (835, 0.17170802083491227),
 (1213, 0.1687738605676036),
 (1366, 0.16204253072828234),
 (278, 0.16171660841618313),
 (2824, 0.15961077078261668),
 (94, 0.15951059071701512),
 (539, 0.15670756319888351),
 (1990, 0.15544505773020806),
 (121, 0.15477265040968

In [19]:
print('Movies suggested for you: \n')

i = 1
for i in range(30):
    index = sorted_similar_movies[i][0]
    title_from_index = movies_df[movies_df.index == index]['title'].values[0]
    print(f'{i} : {title_from_index}')

Movies suggested for you: 

0 : Avatar
1 : X-Men: First Class
2 : Rise of the Planet of the Apes
3 : Dragonball Evolution
4 : Alien
5 : The Watch
6 : Eragon
7 : Fantastic 4: Rise of the Silver Surfer
8 : Live Free or Die Hard
9 : X-Men Origins: Wolverine
10 : Australia
11 : X-Men: The Last Stand
12 : Titanic
13 : The Heat
14 : The Abyss
15 : Alien³
16 : Percy Jackson & the Olympians: The Lightning Thief
17 : Aliens
18 : Sunshine
19 : Wall Street: Money Never Sleeps
20 : True Lies
21 : Life of Pi
22 : The Book of Life
23 : Night at the Museum
24 : Parental Guidance
25 : Gulliver's Travels
26 : Mr. Popper's Penguins
27 : Aliens vs Predator: Requiem
28 : The Devil Wears Prada
29 : Planet of the Apes


In [20]:
# Single Function
def recommend_movie(movie_name):
    list_of_all_titles = movies_df['title'].tolist()
    find_close_matches = difflib.get_close_matches(movie_name, list_of_all_titles)
    close_match = find_close_matches[0]
    idx_of_movie = movies_df[movies_df.title == close_match]['index'].values[0]

    similarity_score = list(enumerate(similarity[idx_of_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    print('Movies suggested for you: \n')

    i = 1
    for i in range(30):
        index = sorted_similar_movies[i][0]
        title_from_index = movies_df[movies_df.index == index]['title'].values[0]
        print(f'{i} : {title_from_index}')


recommend_movie(movie_name = input('Enter your favorite movie name: '))
    

Movies suggested for you: 

0 : Iron Man
1 : Iron Man 2
2 : Iron Man 3
3 : The Avengers
4 : Captain America: Civil War
5 : Avengers: Age of Ultron
6 : Captain America: The Winter Soldier
7 : The Incredible Hulk
8 : Thor
9 : Ant-Man
10 : Captain America: The First Avenger
11 : X-Men: Days of Future Past
12 : Thor: The Dark World
13 : X-Men
14 : The Amazing Spider-Man 2
15 : The Helix... Loaded
16 : X2
17 : Guardians of the Galaxy
18 : X-Men: Apocalypse
19 : Deadpool
20 : Spider-Man 2
21 : X-Men: The Last Stand
22 : The Amazing Spider-Man
23 : X-Men: First Class
24 : Ghost Rider: Spirit of Vengeance
25 : The Wolverine
26 : The Judge
27 : Fantastic Four
28 : X-Men Origins: Wolverine
29 : Made
